# <center>World Map Geovisualization using Folium and Covid Data

### <center>Instructor: Vinita Silaparasetty

### Import Libraries

In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [2]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET", "/summary", payload, headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

### Convert Data to JSON

In [3]:
covid1= json.loads(data)

### Normalize Data 

In [4]:
pd.json_normalize(covid1['Countries'],sep=",")

,ID,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,2ba01d50-ff98-47e3-93cf-f150cd7f3968,Afghanistan,AF,afghanistan,36,56290,2,2469,0,49994,2021-03-28T05:24:32.057Z
1,1d0bcbd2-3adb-4b92-99ca-5746d995bec9,Albania,AL,albania,425,123641,12,2204,550,88899,2021-03-28T05:24:32.057Z
2,efc61394-650e-4f38-86dd-fc2b46fd93a8,Algeria,DZ,algeria,93,116750,3,3077,82,81242,2021-03-28T05:24:32.057Z
3,3aaeba37-3f3f-4cb8-b1c1-18acac873585,Andorra,AD,andorra,77,11809,1,115,23,11172,2021-03-28T05:24:32.057Z
4,89c17e79-6b9d-435f-89c0-53a49a034dcb,Angola,AO,angola,70,22031,1,533,19,20269,2021-03-28T05:24:32.057Z
...,...,...,...,...,...,...,...,...,...,...,...
185,0ab93809-359b-4691-afda-6a1a5eb0313a,Venezuela (Bolivarian Republic),VE,venezuela,0,154905,0,1543,0,143468,2021-03-28T05:24:32.057Z
186,4fc21dab-60e5-4593-b039-68f7a07bd54f,Viet Nam,VN,vietnam,0,2586,0,35,43,2308,2021-03-28T05:24:32.057Z
187,a3c757f4-358f-48f6-babe-e34ae071724e,Yemen,YE,yemen,69,3969,12,832,2,1590,2021-03-28T05:24:32.057Z
188,2c57f843-dafd-4f73-99cf-3f7a3265f001,Zambia,ZM,zambia,0,87583,4,1198,246,84264,2021-03-28T05:24:32.057Z


### Convert to Pandas DataFrame

In [8]:
df = pd.DataFrame(covid1['Countries'])
df.head(3)

,ID,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,2ba01d50-ff98-47e3-93cf-f150cd7f3968,Afghanistan,AF,afghanistan,36,56290,2,2469,0,49994,2021-03-28T05:24:32.057Z,{}
1,1d0bcbd2-3adb-4b92-99ca-5746d995bec9,Albania,AL,albania,425,123641,12,2204,550,88899,2021-03-28T05:24:32.057Z,{}
2,efc61394-650e-4f38-86dd-fc2b46fd93a8,Algeria,DZ,algeria,93,116750,3,3077,82,81242,2021-03-28T05:24:32.057Z,{}


### Drop Unnecessary Columns

In [9]:
covid2 = df.drop(columns =['CountryCode','Slug','Date','Premium'],axis=1)
covid2.head(3)

,ID,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,2ba01d50-ff98-47e3-93cf-f150cd7f3968,Afghanistan,36,56290,2,2469,0,49994
1,1d0bcbd2-3adb-4b92-99ca-5746d995bec9,Albania,425,123641,12,2204,550,88899
2,efc61394-650e-4f38-86dd-fc2b46fd93a8,Algeria,93,116750,3,3077,82,81242


### Generate Base Map

In [11]:
m = folium.Map(tiles="Stamen Terrain", min_zoom=1.5)
m

### Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [12]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

### Generate Choropleth Map Layer

In [13]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='COVID-19',
    data=covid2,
    columns=['Country', 'TotalConfirmed'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Covid Cases',
).add_to(m)
m

### Generate Circular Markers

In [14]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv

In [17]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates.head(3)

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria


In [21]:
covid_final= pd.merge(covid2,coordinates,on='Country')
print(len(covid2))
print(len(covid_final))

190
169


In [22]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                        fill_color='#000000').add_to(m)

In [23]:
covid_final.apply(plotDot, axis = 1)

m.fit_bounds(m.get_bounds())

m

### Generate Base Map

In [24]:
m1 = folium.Map(tiles='StamenToner', min_zoom=2)

m1

In [25]:
deaths=covid_final['TotalDeaths'].astype(float)

In [26]:
lat=covid_final['latitude'].astype(float)

In [27]:
lon=covid_final['longitude'].astype(float)

### Generate Heat Map Layer

In [28]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

### Generate Circular Marker

In [29]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=5,
                        weight=2,
                        popup = [point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)

covid_final.apply(plotDot, axis = 1)

m1.fit_bounds(m1.get_bounds())

m1